<a href="https://colab.research.google.com/github/anosharshad/Practise-Git/blob/master/Gender_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [1]:
import librosa
import IPython.display as ipd
import numpy as np
import pandas as pd
import os

In [ ]:
gender_df = pd.DataFrame(columns=['ID','sex'])

# Picking the data from SPEAKERS.TXT and making dataframe of and ID's and gender

file1 = open('/content/drive/MyDrive/Colab Notebooks/LibriSpeech/SPEAKERS.TXT', 'r') 
Lines = file1.readlines() 

i = 0

for line in Lines:
  if line.startswith(';'):
    pass
  else:

    line = line.replace(" ", "")
    line = line.split('|')
    gender_df.at[i,'ID'] = line[0]
    gender_df.at[i,'sex'] = line[1]

    i = i +1

gender_df.head()

,ID,sex
0,14,F
1,16,F
2,17,M
3,19,F
4,20,F


In [ ]:
import os 

# Picking the files from dev-clean directory and making dataframe with person ID as index and MFCCs coefficients and sex as coulumns for each audio file

Data = pd.DataFrame()

main_dir = os.listdir('/content/drive/MyDrive/Colab Notebooks/LibriSpeech/dev-clean')

# Using the half data 

main_dir = main_dir[0:20]

for id_folder in main_dir:
  id_folder_subfolder = os.listdir('/content/drive/MyDrive/Colab Notebooks/LibriSpeech/dev-clean'+'/'+str(id_folder))

  for data_folder in id_folder_subfolder:
    file_folder = os.listdir('/content/drive/MyDrive/Colab Notebooks/LibriSpeech/dev-clean'+'/'+str(id_folder)+'/'+str(data_folder))
    for file in file_folder:
      if file.endswith('.txt'):
        pass
      else:
        audio_file = '/content/drive/MyDrive/Colab Notebooks/LibriSpeech/dev-clean'+'/'+str(id_folder)+'/'+str(data_folder)+'/'+file
        signal, sr = librosa.load(audio_file)
        mfcc = librosa.feature.mfcc(signal, n_mfcc=20, sr=sr)
        mfccs_processed = np.mean(mfcc.T,axis=0)
        mfccs_processed = mfccs_processed.reshape(1,20)
        gender = gender_df.loc[ (gender_df.ID == id_folder), 'sex'].values[0]
        df = pd.DataFrame(data=mfccs_processed, index=[id_folder])
        df.at[id_folder,'sex'] = gender
        Data = pd.concat([Data, df])


In [ ]:
Data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,sex
174,-501.744238,109.983814,-24.246647,34.718577,-21.044219,16.188451,-5.204373,0.634877,-13.175152,-2.324837,0.679354,-5.524442,-3.602958,-7.844760,6.484891,-3.430486,1.416500,-5.013810,1.173393,-3.595596,M
174,-523.684890,127.009969,-12.127082,10.799136,-9.631838,15.317657,5.412192,-3.754034,-8.419761,2.470246,-0.758966,-3.187594,-1.508664,-7.838356,1.293137,-1.661352,4.314133,-1.583486,-1.072215,1.871589,M
174,-402.545096,111.973975,-29.493962,25.502694,-24.122551,16.134334,-1.890146,-4.369007,-10.646006,-5.036118,-5.070844,-7.759259,-3.112405,-6.994991,4.757550,-3.191289,0.463054,-5.376310,-0.676222,-0.591905,M
174,-452.032887,131.016168,-25.497588,13.440026,-23.046601,14.327636,2.688842,-3.744222,-7.970796,1.391805,-2.579652,-5.768270,-1.916512,-6.478376,3.554020,-2.287034,0.606678,-8.299305,-2.577945,-0.141906,M
174,-494.896028,86.896507,-18.297764,31.144922,-12.778350,8.206793,-0.938327,-2.933934,-4.673850,2.100469,-1.546953,-2.346628,-0.011496,-5.861083,0.248961,-5.817758,1.576846,-5.082866,-2.616495,2.547774,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2902,-410.231737,99.284289,-10.742460,24.169037,-9.087096,7.275328,-26.427665,-15.814995,-17.143097,-2.502969,-3.166967,-2.608572,6.153468,-11.232628,4.412489,-0.943399,-8.447776,-11.106552,-1.361228,-1.229088,M
2902,-377.721994,91.995554,-5.075856,20.866581,-10.992234,-3.119264,-25.534643,-15.458302,-16.934683,-6.017462,-4.258823,-0.170763,7.902674,-10.893585,4.087962,1.856620,-10.160096,-11.321085,-2.340341,-1.300948,M
2902,-359.017806,92.268295,-10.924649,21.348452,-19.393815,15.866940,-35.830224,-4.539294,-14.979179,-3.746499,7.841660,0.491391,3.412194,-9.875238,4.237035,-1.075362,-1.636346,-14.494286,-0.378434,-1.089240,M
2902,-384.535418,103.659889,-14.237995,26.690230,-13.467905,2.250127,-32.700679,-15.407890,-17.059905,-0.087401,-2.572432,-3.961692,7.877682,-10.710061,5.636664,-2.912524,-11.278957,-11.393809,-0.643425,-2.668074,M


In [ ]:
Data.to_csv(r'/content/drive/MyDrive/Colab Notebooks/data.csv', index = False)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPool1D

from tensorflow.keras.optimizers import Adam

In [6]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data.csv') 
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,sex
0,-501.744238,109.983814,-24.246647,34.718577,-21.044219,16.188451,-5.204373,0.634877,-13.175152,-2.324837,0.679354,-5.524442,-3.602958,-7.844760,6.484891,-3.430486,1.416500,-5.013810,1.173393,-3.595596,M
1,-523.684890,127.009969,-12.127082,10.799136,-9.631838,15.317657,5.412192,-3.754034,-8.419761,2.470246,-0.758966,-3.187594,-1.508664,-7.838356,1.293137,-1.661352,4.314133,-1.583486,-1.072215,1.871589,M
2,-402.545096,111.973975,-29.493962,25.502694,-24.122551,16.134334,-1.890146,-4.369007,-10.646006,-5.036118,-5.070844,-7.759259,-3.112405,-6.994991,4.757550,-3.191289,0.463054,-5.376310,-0.676222,-0.591905,M
3,-452.032887,131.016168,-25.497588,13.440026,-23.046601,14.327636,2.688842,-3.744222,-7.970796,1.391805,-2.579652,-5.768270,-1.916512,-6.478376,3.554020,-2.287034,0.606678,-8.299305,-2.577945,-0.141906,M
4,-494.896028,86.896507,-18.297764,31.144922,-12.778350,8.206793,-0.938327,-2.933934,-4.673850,2.100469,-1.546953,-2.346628,-0.011496,-5.861083,0.248961,-5.817758,1.576846,-5.082866,-2.616495,2.547774,M


In [7]:
# convert srting data into numberic eg. male 1, female 0
df1 = data
df1 = df1.drop(['sex'],axis = 1)
X1 = df1.values
y1 = data['sex'].values

# only one column has object type so I encode it

encoder = LabelEncoder()
y1 = encoder.fit_transform(y1)

In [8]:
scaler = StandardScaler()
scaler.fit(X1)
X1 = scaler.transform(X1)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, random_state=1)

In [10]:
# SVC

from sklearn.svm import SVC
from sklearn import metrics
svc=SVC() #Default hyperparameters
svc.fit(X_train,y_train)
y_pred1=svc.predict(X_test)
print(metrics.accuracy_score(y_test,y_pred1))

0.9923664122137404


In [11]:
# KNeighborsClassifier (KNN)

from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()
clf.fit(X_train, y_train)
y_pred2 = clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred2))

1.0


In [12]:
#Random forest 

rand_forest = RandomForestClassifier()
rand_forest.fit(X_train, y_train)
y_pred3 = rand_forest.predict(X_test)
print(metrics.accuracy_score(y_test,y_pred3))

0.9732824427480916


In [13]:
# Deep neural network (CNN)
data.shape

(1309, 21)

In [14]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,sex
0,-501.744238,109.983814,-24.246647,34.718577,-21.044219,16.188451,-5.204373,0.634877,-13.175152,-2.324837,0.679354,-5.524442,-3.602958,-7.844760,6.484891,-3.430486,1.416500,-5.013810,1.173393,-3.595596,M
1,-523.684890,127.009969,-12.127082,10.799136,-9.631838,15.317657,5.412192,-3.754034,-8.419761,2.470246,-0.758966,-3.187594,-1.508664,-7.838356,1.293137,-1.661352,4.314133,-1.583486,-1.072215,1.871589,M
2,-402.545096,111.973975,-29.493962,25.502694,-24.122551,16.134334,-1.890146,-4.369007,-10.646006,-5.036118,-5.070844,-7.759259,-3.112405,-6.994991,4.757550,-3.191289,0.463054,-5.376310,-0.676222,-0.591905,M
3,-452.032887,131.016168,-25.497588,13.440026,-23.046601,14.327636,2.688842,-3.744222,-7.970796,1.391805,-2.579652,-5.768270,-1.916512,-6.478376,3.554020,-2.287034,0.606678,-8.299305,-2.577945,-0.141906,M
4,-494.896028,86.896507,-18.297764,31.144922,-12.778350,8.206793,-0.938327,-2.933934,-4.673850,2.100469,-1.546953,-2.346628,-0.011496,-5.861083,0.248961,-5.817758,1.576846,-5.082866,-2.616495,2.547774,M


In [15]:
for index in data.index:
  temp = data.loc[index,'sex']
  if temp == 'M':
    data.at[index,'sex'] = int(1)
  if temp == 'F':
    data.at[index,'sex'] = int(0)
  

data.sex = data.sex.astype('int64') 

In [16]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,sex
0,-501.744238,109.983814,-24.246647,34.718577,-21.044219,16.188451,-5.204373,0.634877,-13.175152,-2.324837,0.679354,-5.524442,-3.602958,-7.844760,6.484891,-3.430486,1.416500,-5.013810,1.173393,-3.595596,1
1,-523.684890,127.009969,-12.127082,10.799136,-9.631838,15.317657,5.412192,-3.754034,-8.419761,2.470246,-0.758966,-3.187594,-1.508664,-7.838356,1.293137,-1.661352,4.314133,-1.583486,-1.072215,1.871589,1
2,-402.545096,111.973975,-29.493962,25.502694,-24.122551,16.134334,-1.890146,-4.369007,-10.646006,-5.036118,-5.070844,-7.759259,-3.112405,-6.994991,4.757550,-3.191289,0.463054,-5.376310,-0.676222,-0.591905,1
3,-452.032887,131.016168,-25.497588,13.440026,-23.046601,14.327636,2.688842,-3.744222,-7.970796,1.391805,-2.579652,-5.768270,-1.916512,-6.478376,3.554020,-2.287034,0.606678,-8.299305,-2.577945,-0.141906,1
4,-494.896028,86.896507,-18.297764,31.144922,-12.778350,8.206793,-0.938327,-2.933934,-4.673850,2.100469,-1.546953,-2.346628,-0.011496,-5.861083,0.248961,-5.817758,1.576846,-5.082866,-2.616495,2.547774,1


In [17]:
X = data.drop(['sex'], axis=1)
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-501.744238,109.983814,-24.246647,34.718577,-21.044219,16.188451,-5.204373,0.634877,-13.175152,-2.324837,0.679354,-5.524442,-3.602958,-7.844760,6.484891,-3.430486,1.416500,-5.013810,1.173393,-3.595596
1,-523.684890,127.009969,-12.127082,10.799136,-9.631838,15.317657,5.412192,-3.754034,-8.419761,2.470246,-0.758966,-3.187594,-1.508664,-7.838356,1.293137,-1.661352,4.314133,-1.583486,-1.072215,1.871589
2,-402.545096,111.973975,-29.493962,25.502694,-24.122551,16.134334,-1.890146,-4.369007,-10.646006,-5.036118,-5.070844,-7.759259,-3.112405,-6.994991,4.757550,-3.191289,0.463054,-5.376310,-0.676222,-0.591905
3,-452.032887,131.016168,-25.497588,13.440026,-23.046601,14.327636,2.688842,-3.744222,-7.970796,1.391805,-2.579652,-5.768270,-1.916512,-6.478376,3.554020,-2.287034,0.606678,-8.299305,-2.577945,-0.141906
4,-494.896028,86.896507,-18.297764,31.144922,-12.778350,8.206793,-0.938327,-2.933934,-4.673850,2.100469,-1.546953,-2.346628,-0.011496,-5.861083,0.248961,-5.817758,1.576846,-5.082866,-2.616495,2.547774


In [18]:
y = data.sex.values

In [19]:
y

array([1, 1, 1, ..., 1, 1, 1])

In [21]:
X.shape

(1309, 20)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)

In [23]:
X_train.shape

(1047, 20)

In [24]:
X_test.shape

(262, 20)

In [25]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [26]:
X_train = X_train.reshape(1047,20,1)
X_test = X_test.reshape(262, 20, 1)

In [27]:
epochs = 50
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape = (20,1)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv1D(filters=64, kernel_size=2, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

In [28]:
model.compile(optimizer=Adam(lr=0.00005), loss = 'binary_crossentropy', metrics=['accuracy'])

In [29]:
history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=1)

Epoch 1/50
33/33 [==============================] - 0s 13ms/step - loss: 1.1328 - accuracy: 0.5750 - val_loss: 0.6629 - val_accuracy: 0.6985
Epoch 2/50
33/33 [==============================] - 0s 5ms/step - loss: 0.9892 - accuracy: 0.5759 - val_loss: 0.6338 - val_accuracy: 0.7901
Epoch 3/50
33/33 [==============================] - 0s 5ms/step - loss: 0.8391 - accuracy: 0.6609 - val_loss: 0.6006 - val_accuracy: 0.8740
Epoch 4/50
33/33 [==============================] - 0s 5ms/step - loss: 0.7102 - accuracy: 0.6972 - val_loss: 0.5626 - val_accuracy: 0.8702
Epoch 5/50
33/33 [==============================] - 0s 6ms/step - loss: 0.6737 - accuracy: 0.7202 - val_loss: 0.5185 - val_accuracy: 0.8702
Epoch 6/50
33/33 [==============================] - 0s 5ms/step - loss: 0.5653 - accuracy: 0.7526 - val_loss: 0.4709 - val_accuracy: 0.8664
Epoch 7/50
33/33 [==============================] - 0s 5ms/step - loss: 0.4610 - accuracy: 0.7947 - val_loss: 0.4206 - val_accuracy: 0.8740
Epoch 8/50
33/33 [=

In [30]:
history.history

{'accuracy': [0.5749761462211609,
  0.5759312510490417,
  0.6609359979629517,
  0.6972301602363586,
  0.7201527953147888,
  0.7526265382766724,
  0.7946513891220093,
  0.7927411794662476,
  0.8156638145446777,
  0.8147087097167969,
  0.8233046531677246,
  0.8452721834182739,
  0.8576886057853699,
  0.8576886057853699,
  0.8576886057853699,
  0.873925507068634,
  0.8806112408638,
  0.8911174535751343,
  0.8930277228355408,
  0.8997134566307068,
  0.8949379324913025,
  0.9092645645141602,
  0.8987583518028259,
  0.9121298789978027,
  0.915950357913971,
  0.9188156723976135,
  0.9178605675697327,
  0.9264565706253052,
  0.9255014061927795,
  0.9388729929924011,
  0.9235911965370178,
  0.9321871995925903,
  0.9283667802810669,
  0.9407832026481628,
  0.9264565706253052,
  0.934097409248352,
  0.9331423044204712,
  0.9379178881645203,
  0.9379178881645203,
  0.9283667802810669,
  0.9474689364433289,
  0.9388729929924011,
  0.9531996250152588,
  0.9379178881645203,
  0.9446036219596863,
  0.